In [ ]:
cd /content/drive/MyDrive/20210611任务

/content/drive/MyDrive/20210611任务


In [ ]:
!pip install transformers sentencepiece datasets rouge_score

     |████████████████████████████████| 2.9 MB 7.5 MB/s 
     |████████████████████████████████| 1.2 MB 55.2 MB/s 
     |████████████████████████████████| 270 kB 43.3 MB/s 
     |████████████████████████████████| 895 kB 59.9 MB/s 
     |████████████████████████████████| 3.3 MB 39.9 MB/s 
     |████████████████████████████████| 636 kB 63.6 MB/s 
     |████████████████████████████████| 52 kB 1.8 MB/s 
     |████████████████████████████████| 123 kB 68.1 MB/s 
     |████████████████████████████████| 1.3 MB 15.9 MB/s 
     |████████████████████████████████| 243 kB 60.0 MB/s 
     |████████████████████████████████| 294 kB 72.6 MB/s 
     |████████████████████████████████| 142 kB 75.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!wget -c http://nlp.stanford.edu/data/glove.840B.300d.zip

--2021-09-22 16:00:43--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2021-09-22 16:00:43--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip [following]
--2021-09-22 16:00:43--  http://downloads.cs.stanford.edu/nlp/data/glove.840B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file

In [ ]:
import zipfile
with zipfile.ZipFile("./glove.840B.300d.zip", "r") as zipread:
  zipread.extractall("./W2Vmodel/")
  zipread.close

In [ ]:
import numpy as np
import pandas as pd
import nltk
import spacy
import os
import string
import gensim
from gensim.models import Word2Vec
from gensim.scripts.glove2word2vec import glove2word2vec

In [ ]:

PATH_TO_GLOVE = "./W2Vmodel/glove.840B.300d.txt"
tmp_file = "./glove.840B.300d.w2v.txt"

glove2word2vec(PATH_TO_GLOVE, tmp_file)
glove = gensim.models.KeyedVectors.load_word2vec_format(tmp_file)

In [ ]:
import scipy
from ast import literal_eval
def cosine_distance_wordembedding_method(s1, s2):
    vector_1 = np.mean([glove[word] for word in s1 if word in glove],axis=0)
    vector_2 = np.mean([glove[word] for word in s2 if word in glove],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return 1-cosine

In [ ]:
data = pd.read_csv("./MiniMatching.csv")
data

,Unnamed: 0,real_target,QS-Prediction,AS-Prediction
0,0,meningoencephalitis : causes,meningoencephalitis is a chronic disease . .,meningoencephalitis : physiology : causes phys...
1,1,ingrown nail : prevention,ingrown nail : prevention . click here for mor...,ingrown toe nails : physiology . diagnosis : c...
2,2,hives : symptoms,hives symptoms : symptoms of a coma .,cutaneous condition|welts from hives . causes ...
3,3,central diabetes insipidus : treatments,central diabetes insipidus is a disease charac...,desmopressin : treatments physiology : treatme...
4,4,amnesia : general,amnesia is the most populous country in the wo...,amnesia : physiology : symptoms : causes phys...
...,...,...,...,...
412,412,subclavian steal syndrome : physiology,subclavian steal syndrome : physiology . .,sss : physiology : symptoms : causes . hepatit...
413,413,schistosoma japonicum : prevention,schistosoma japonicum : prevention .,schistosomiasis : physiology - prevention : tr...
414,414,cutaneous leishmaniasis : symptoms,cutaneous leishmaniasis . symptoms : symptoms .,post kala-azar dermal leishmaniasis : physiolo...
415,415,coagulopathy : treatments,coagulopathy is a treatment for a symptom .,coagulopathy : treatments physiology - treatme...


In [ ]:
new_QS = []
for line in data['S_Q']:
  new_QS.append(line.replace(" : "," ").split(" "))
new_AS = []
for line in data['S_A']:
  new_AS.append(line.replace(" : "," ").split(" "))

In [ ]:
data['QS'] = new_QS
data['AS'] = new_AS

In [ ]:
data

,Unnamed: 0,real_target,QS-Prediction,AS-Prediction,QS,AS
0,0,meningoencephalitis : causes,meningoencephalitis is a chronic disease . .,meningoencephalitis : physiology : causes phys...,"[meningoencephalitis, is, a, chronic, disease,...","[meningoencephalitis, physiology, causes, phys..."
1,1,ingrown nail : prevention,ingrown nail : prevention . click here for mor...,ingrown toe nails : physiology . diagnosis : c...,"[ingrown, nail, prevention, ., click, here, fo...","[ingrown, toe, nails, physiology, ., diagnosis..."
2,2,hives : symptoms,hives symptoms : symptoms of a coma .,cutaneous condition|welts from hives . causes ...,"[hives, symptoms, symptoms, of, a, coma, .]","[cutaneous, condition|welts, from, hives, ., c..."
3,3,central diabetes insipidus : treatments,central diabetes insipidus is a disease charac...,desmopressin : treatments physiology : treatme...,"[central, diabetes, insipidus, is, a, disease,...","[desmopressin, treatments, physiology, treatme..."
4,4,amnesia : general,amnesia is the most populous country in the wo...,amnesia : physiology : symptoms : causes phys...,"[amnesia, is, the, most, populous, country, in...","[amnesia, physiology, symptoms, causes, , phys..."
...,...,...,...,...,...,...
412,412,subclavian steal syndrome : physiology,subclavian steal syndrome : physiology . .,sss : physiology : symptoms : causes . hepatit...,"[subclavian, steal, syndrome, physiology, ., .]","[sss, physiology, symptoms, causes, ., hepatit..."
413,413,schistosoma japonicum : prevention,schistosoma japonicum : prevention .,schistosomiasis : physiology - prevention : tr...,"[schistosoma, japonicum, prevention, .]","[schistosomiasis, physiology, -, prevention, t..."
414,414,cutaneous leishmaniasis : symptoms,cutaneous leishmaniasis . symptoms : symptoms .,post kala-azar dermal leishmaniasis : physiolo...,"[cutaneous, leishmaniasis, ., symptoms, sympto...","[post, kala-azar, dermal, leishmaniasis, physi..."
415,415,coagulopathy : treatments,coagulopathy is a treatment for a symptom .,coagulopathy : treatments physiology - treatme...,"[coagulopathy, is, a, treatment, for, a, sympt...","[coagulopathy, treatments, physiology, -, trea..."


In [ ]:
new_list = []
for line in data['S_Q']:
  text = line.replace(" : "," ").split(" ")
  for i in range(417):
    new_list.append(text)
new_list2 = []
for i in range(417):
  for line in data['S_A']:
    new_list2.append(line.replace(" : "," ").split(" "))
new_list3 = []
for i in range(417*417):
  if i % 417 == 0:
    new_list3.append(1)
  else:
    new_list3.append(0)

In [ ]:
new_list = []
new_list2 = []
new_list3 = []

new_LHS_Q = []
new_LHS_A = []
for i, line in enumerate(data['S_Q']):
  text = line.replace(" : "," ").split(" ")
  new_list.append(text)
  new_list2.append(data['S_A'][i])
  new_list3.append(1)
  new_LHS_Q.append(data['LHS_Q'][i])
  new_LHS_A.append(data['LHS_A'][i])
  for _ in range(5):
    temp = random.randint(0,416)
    if temp != i:
      new_list.append(text)
      new_list2.append(data['S_A'][temp])
      new_list3.append(0)
      new_LHS_Q.append(data['LHS_Q'][temp])
      new_LHS_A.append(data['LHS_A'][temp])


In [ ]:
import numpy as np
def cosine_similarity(x,y):
  x = np.array(x).reshape([-1])
  y = np.array(y).reshape([-1])
  num = x.dot(y.T)
  denom = np.linalg.norm(x) * np.linalg.norm(y)
  return num / denom
# data["LHS_cos"] = data.apply(lambda x: cosine_similarity(x["LHS_Q"], x["LHS_A"]), axis=1)
# data

In [ ]:
import scipy
from ast import literal_eval
def cosine_distance_wordembedding_method(s1, s2):
    vector_1 = np.mean([glove[word] for word in s1 if word in glove],axis=0)
    vector_2 = np.mean([glove[word] for word in s2 if word in glove],axis=0)
    cosine = scipy.spatial.distance.cosine(vector_1, vector_2)
    return 1-cosine
# data["cos_sim"] = data.apply(lambda x: cosine_distance_wordembedding_method(x["QS-Prediction"], x["AS-Prediction"]), axis=1)
# data

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from scipy.linalg import norm
def tf_similarity(s1, s2):
  def add_space(s):
    return ' '.join(list(s))
  
  # 将字中间加入空格
  s1, s2 = add_space(s1), add_space(s2)
  # 转化为TF矩阵
  cv = CountVectorizer(tokenizer=lambda s: s.split())
  corpus = [s1, s2]
  vectors = cv.fit_transform(corpus).toarray()
  # 计算TF系数
  return np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1]))
# data["TF"] = data.apply(lambda x: tf_similarity(x["QS-Prediction"], x["AS-Prediction"]), axis=1)
# data

In [ ]:
new_data = pd.DataFrame({'QS':new_list,'AS':new_list2,'true':new_list3,"LHS_Q":new_LHS_Q,"LHS_A":new_LHS_Q})
new_data["cos_sim"] = new_data.apply(lambda x: cosine_distance_wordembedding_method(x.QS, x.AS), axis=1)
new_data["TF"] = new_data.apply(lambda x: tf_similarity(x["QS-Prediction"], x["AS-Prediction"]), axis=1)
new_data["LHS_cos"] = new_data.apply(lambda x: cosine_similarity(x["LHS_Q"], x["LHS_A"]), axis=1)
new_data

In [ ]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(new_data["cos_sim","LHS_Cos","TF"], new_data["true"],test_size=0.3)

In [ ]:
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

options = {"model_param":
         {"eta": 0.015, 
         "max_depth": 5,
         "n_estimators":20,
         "sub_sample":0.5,
         "scale_pos_weight":1.75,
         "random_state":2,
         "eval_metric": "logloss",
         "objective": "binary:logistic"},
         "num_round":50,
         "test_frac":0.2}

In [ ]:
#training model
xgbmatrix = xgb.DMatrix(train_X, train_y)
xgb_model = xgb.train(options["model_param"], xgbmatrix, options["num_round"], verbose_eval=10)
xgb_valmatrix = xgb.DMatrix(val_X)
y_val_pred = xgb_model.predict(xgb_valmatrix)
gsearch = GridSearchCV(xgb_model , param_grid = param_test, scoring='roc_auc', cv=5 )

# model = XGBClassifier(max_depth=2,learning_rate=0.01,
# n_estimators=128,
# reg_lambda=0.000001,
# reg_alpha=0.0001)
# model.fit(train_X, train_y)
# pre = model.predict(val_X)

In [ ]:
y_val_pred_binary = [1 if y>0.38 else 0 for y in y_val_pred]
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
accuracy_score(val_y, y_val_pred_binary)

0.7129506008010681

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(val_y, pre)

0.8317757009345794

0.7803073065148262